<a href="https://colab.research.google.com/github/windinherhair/fs_co_tr/blob/master/villain_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# deepfakes/faceswap (villain_256モデル)

## **（重要）colabの12時間&90分ルール対策**

1.新しいインスタンスを起動してから12時間経つと全てリセットされる。（GoogleDriveに学習したmodelや抽出した顔を退避させます）

2.90分アイドル状態が続くとノートの接続が外れるため、** Chromeのアドオンを使って60分置きにページをリロードする。（[Auto Refresh](https://chrome.google.com/webstore/detail/auto-refresh/ifooldnmmcmlbdennkpdnlnbgbmfalko)）**

放置する場合は
1、Auto Refreshをオン
2、手動でノートを保存
3、手動で更新。(保存うんぬんのポップでたら、→はい)
4、colabのタブを開いた状態で放置

がよさそうです。



### 事前準備
1.基本的にextractとconvertはローカルでやって、DRIVEで同期するのがおすすめです。


---


以下、最初からやる場合

2.体側の人の2分くらいの動画（videoA.mp4）、洗練されたBの顔セット（alignB）フォルダを用意する。（なければなるべく同じ色調、角度の顔側の人の動画10分くらい（videoB.mp4）を用意する。）※()内の名前にリネームしてください。

3.Google DriveにvideoA.mp4、alignBフォルダ、(なければvideoB.mp4）をアップロード


4.(推奨)学習済みmodelがある場合は、modelフォルダをGoogleDriveにアップロード

### その他
コメントアウト（#）をコードの前につけるとコードを呼び飛ばします。先に自分の条件に合わせてつけたり外したりしてから実行してください。
また、二回目からはDriveにalign(img)ができてるので、ffmpegとextractに(#)つけて、trainからやって下さい。


### **（設定）ランタイムのタイプをGPUに変更する** 

---



準備が整ったら、

**ランタイム→すべてのセルを実行**


---





In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
# インスタンスが作られてからの日数表示。0.5days=12時間まで使える。

**Google Driveをマウント**（※ここだけ要入力）


###  URLをクリックし、そのまま進めたら出てくるコードをコピー、枠にペーストしEnter




In [0]:
from google.colab import drive
drive.mount('/content/drive')

Githubのdeepfakes/faceswapのtrain-refactorブランチをクローン

In [0]:
!git clone https://github.com/deepfakes/faceswap.git

In [0]:
!cp -r /content/drive/'My Drive'/faceswap /content

faceswapディレクトリに移動

In [0]:
cd faceswap

残りの必要なものをインストール

In [0]:
!pip install -r requirements.txt*

In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
# インスタンスが作られてからの日数表示。0.5days=12時間まで使える。

## **顔セットをZIPから解凍**
#注意）一番最初にここをセットしてから実行する

In [0]:
!unzip /content/drive/'My Drive'/aligned_neru.zip

In [0]:
!unzip /content/drive/'My Drive'/faceA_nano.zip

In [0]:
#!rm -r faceA

In [0]:
#!rm -r aligned

In [0]:
#ls /content/drive/'My Drive'/



---


## (選択)**Train**(バッチ32で100回ごとに保存)※設定はお好みで

プレビューが見れないので6時間くらいを目安にやってみてください。※事前学習済みmodelがある前提です。

コメントアウト(#)外して使ってください

In [0]:
!python faceswap.py train -A /content/faceswap/faceA -B /content/faceswap/aligned -m /content/drive/'My Drive'/model_256/ -s 150 -t villain -bs 16 -wl

In [0]:
#!python faceswap.py train -A /content/faceswap/faceA -B /content/faceswap/aligned -m /content/drive/'My Drive'/model_256/ -s 150 -t villain -bs 8 -w -wl

In [0]:
!cp config/train.ini /content/drive/'My Drive'/faceswap/config/

In [0]:
#ls /content/drive/'My Drive'/model_256

In [0]:
#!python faceswap.py train -A /content/faceswap/alignedHS -B /content/faceswap/alignedHS -m /content/drive/'My Drive'/model_224/ -s 2000 -t OriginalHighRes -bs 32 

In [0]:
#villain
#!python faceswap.py train -A /content/faceswap/faceA -B /content/faceswap/aligned -m /content/drive/'My Drive'/model_vill/ -s 100 -t original-villain -bs 16 

In [0]:
#オプションはお好みで
#!python faceswap.py train -h

#学習が終わったらモデルをダウンロード



---


**Convert**



In [0]:
#!python faceswap.py convert -i /content/drive/'My Drive'/imgA/ -o /content/drive/'My Drive'/mergeAz -al /content/drive/'My Drive'/imgA/alignments.json -m /content/drive/'My Drive'/model/ -a /content/drive/'My Drive'/alignA/ -t villain -c Masked -b 30 -M facehull

In [0]:
#オプションはお好みで
#!python faceswap.py convert -h



---

ffmpegで画像から動画（merged_videoA.mp4）に変換

In [0]:
#!ffmpeg -framerate 29.97 -i /content/drive/'My Drive'/mergeA/imgA%06d.png -vcodec libx264 -pix_fmt yuv420p -r 29.97 /content/drive/'My Drive'/merged_videoA1.mp4

In [0]:
#ls /content/drive/'My Drive'/mergeA/

In [0]:
#cp -u /content/drive/'My Drive'/imgA/*.png /content/drive/'My Drive'/mergeA/

# 終了～